Login to Chameleon and download openrc.sh file from [here](https://testbed.expeca.proj.kth.se/project/api_access/openrc/). Upload it here next to this notebook and continue.

In the next cell, we setup the authentication method to be able to use Openstack clients.

In [ ]:
import os, re
from getpass import getpass

with open('nils-project-openrc.sh', 'r') as f:
    script_content = f.read()
    pattern = r'export\s+(\w+)\s*=\s*("[^"]+"|[^"\n]+)'
    matches = re.findall(pattern, script_content)

    for name, value in matches:
        os.environ[name] = value.strip('"')

# password read from file

Install required packages and dependencies. Ignore the warnings.

In [ ]:
!pip uninstall -q -y moviepy
!pip install -q jedi
!pip install -q git+https://github.com/KTH-EXPECA/python-chi

Import packages

In [ ]:
import json
from loguru import logger
import chi.network, chi.container
from chi.expeca import reserve, list_reservations, unreserve_byid, get_container_status, wait_until_container_removed, get_available_publicips, get_worker_interfaces

Reserve the required equipment and resources

In [ ]:
experiment_duration = {"days": 1, "hours": 0}

# List of required leases with details
required_leases = [
    { 
        "type": "network",
        "name": "ep5g",
        "net_name": "ep5g-vip",
        "segment_id": "100",
        "duration": experiment_duration
    },
    {
        "type": "network",
        "name": "adv-04",
        "net_name": "adv-04",
        "segment_id": "134",
        "duration": experiment_duration,
    },
    {
        "type": "device",
        "name": "worker-08",
        "duration": experiment_duration,
    },
    {
        "type": "device",
        "name": "worker-03",
        "duration": experiment_duration,
    },
]

# List of previously existing leases
existing_leases = list_reservations(brief=True)

# Reserve outstanding resources
for required_lease in required_leases:
    lease_name_with_suffix = required_lease["name"] + "-lease"
    is_already_leased = any(
        existing_lease["name"] == lease_name_with_suffix 
        and existing_lease["status"] == "ACTIVE" 
        for existing_lease in existing_leases
    )
    if is_already_leased:
        logger.info(f"Resource {required_lease['name']} is already leased.")
        continue
    else:
        reserve(required_lease)

def get_reservation_id_by_name(name):
    for lease in list_reservations(brief=True):
        if name in lease['name']:
            return lease['reservation_id']

Create networks

In [ ]:
# Create nils-edgenet for talker
try: 
    edgenet = chi.network.get_network("nils-edgenet")
    edgenet_subnet = chi.network.get_subnet("nils-edgenet-subnet")
    logger.info("nils-edgenet already exists.")
except:
    edgenet = chi.network.create_network("nils-edgenet")
    edgenet_subnet = chi.network.create_subnet(
        "nils-edgenet-subnet", 
        edgenet["id"], 
        "10.70.70.0/24", 
        gateway_ip="10.70.70.1", 
        enable_dhcp=False
    )
    logger.success("nils-edgenet is created.")

Create routers

In [ ]:
ep5gnet = chi.network.get_network("ep5g-vip-net")

try:
    chi.network.get_router("nils-router")
    logger.info("Router already exists.")
except:
    router = chi.network.create_router("nils-router", "public")
    chi.network.add_subnet_to_router(router["id"], edgenet_subnet["id"])
    chi.network.add_subnet_to_router(router["id"], ep5gnet["subnets"][0])
    chi.network.add_route_to_router(router["id"],"172.16.0.0/16","10.30.111.10")
    logger.success("Router created and subnets added.")

Project settings

In [ ]:
discovery_addr = "10.70.70.80"
talker_addr = "10.70.70.90"
listener_addr = "10.42.3.2"
listener_nat_addr = "172.16.0.96"

discovery_addr_l2tp = "172.18.0.2"
listener_addr_l2tp = "172.18.0.3"
broadcast_addr_l2tp = "172.18.0.255"

discovery_worker ="worker-03"
talker_worker = "worker-03"
listener_worker = "worker-08"

Start discovery-server

In [ ]:
discovery_container_name = "discovery-server"

try:
    chi.container.destroy_container(discovery_container_name)
    wait_until_container_removed(discovery_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

public_net = chi.network.get_network("serverpublic")

# check public IPs and select one
available_ips = get_available_publicips()
discovery_addr_public = available_ips[-1]
logger.info(f"Public IP address is {discovery_addr_public} for this container.")

discovery_image_name = "nilsjor/ros-humble-turtlebot:discovery-v0.8"

discovery_env_vars = {
    "ROS_DISCOVERY_SERVER":"localhost:11811",
    "DNS_IP":"1.1.1.1",
    "GATEWAY_IP":"130.237.11.97",
    "PASS":"turtlebot",
    "L2TP_LOCAL_IP":discovery_addr,
    "L2TP_REMOTE_IP":listener_nat_addr,
    "L2TP_ENDPOINT_CIDR":f"{discovery_addr_l2tp}/24",
    "L2TP_BROADCAST":broadcast_addr_l2tp,
}

discovery_labels = {
    "networks.1.interface":"eno12399np0",
    "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    "networks.1.ip":f"{discovery_addr}/24",
    "networks.2.interface":"ens1f1",
    "networks.2.ip":f"{discovery_addr_public}/27",
    "networks.2.gateway":"130.237.11.97",
    "capabilities.privileged":"true",
}

discovery_container = chi.container.create_container(
    name = discovery_container_name,
    image = discovery_image_name,
    reservation_id = get_reservation_id_by_name(discovery_worker),
    environment = discovery_env_vars,
    nets = [
        { "network" : edgenet['id'] },
        { "network" : public_net['id'] },
    ],
    labels = discovery_labels,
)
chi.container.wait_for_active(discovery_container_name)
logger.success("Container deployed and active.")

Start talker

In [ ]:
talker_image_name = "nilsjor/ros-humble-turtlebot:talker-v0.5"

talker_env_vars = {
    "ROS_DISCOVERY_SERVER":f"{discovery_addr}:11811",
}

talker_labels = {
    "networks.1.interface":"eno12409np1",
    "networks.1.routes":"172.16.0.0/16-10.70.70.1",
    "networks.1.ip":f"{talker_addr}/24",
}

talker_container_name = "talker"

try:
    chi.container.destroy_container(talker_container_name)
    wait_until_container_removed(talker_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

talker_container = chi.container.create_container(
    name = talker_container_name,
    image = talker_image_name,
    reservation_id = get_reservation_id_by_name(talker_worker),
    environment = talker_env_vars,
    nets = [
        { "network" : edgenet['id'] },
    ],
    labels = talker_labels,
)

chi.container.wait_for_active(talker_container_name)
logger.success("Container deployed and active.")

Start listener

In [ ]:
devicenet = chi.network.get_network("adv-04-net")

listener_image_name = "nilsjor/ros-humble-turtlebot:listener-v0.7"

listener_env_vars = {
    "ROS_DISCOVERY_SERVER":f"{discovery_addr_l2tp}:11811",
    "L2TP_LOCAL_IP":listener_addr, #TODO: test with listener_nat_addr as well...
    "L2TP_REMOTE_IP":discovery_addr,
    "L2TP_ENDPOINT_CIDR":f"{listener_addr_l2tp}/24",
    "L2TP_BROADCAST":broadcast_addr_l2tp,
}

listener_labels = {
    "networks.1.interface":"eno12409",
    "networks.1.ip":f"{listener_addr}/24",
    "networks.1.routes":"10.70.70.0/24-10.42.3.1",
    "capabilities.privileged":"true",
}

listener_container_name = "listener"

try:
    chi.container.destroy_container(listener_container_name)
    wait_until_container_removed(listener_container_name)
    logger.success("Previous container destroyed.")
except:
    logger.info("No previous container found.")

listener_container = chi.container.create_container(
    name = listener_container_name,
    image = listener_image_name,
    reservation_id = get_reservation_id_by_name(listener_worker),
    environment = listener_env_vars,
    nets = [
        { "network" : devicenet['id'] },
    ],
    labels = listener_labels,
)

chi.container.wait_for_active(listener_container_name)
logger.success("Container deployed and active.")

In [ ]:
command = "tcpdump -i net1"
result = chi.container.execute(
    container_ref=discovery_container_name,
    command="curl -s -X POST -H \"Content-Type: application/json\" -d '{\"cmd\": \"" + command + "\"}' http://localhost:50505/",
)
logger.info(f"{result}")

## Teardown

Destroy all containers

In [ ]:
try:
    status = get_container_status(discovery_container_name)
    chi.container.destroy_container(discovery_container_name)
    wait_until_container_removed(discovery_container_name)
except:
    logger.info("No discovery container found.")

try:
    status = get_container_status(talker_container_name)
    chi.container.destroy_container(talker_container_name)
    wait_until_container_removed(talker_container_name)
except:
    logger.info("No talker container found.")

try:
    status = get_container_status(listener_container_name)
    chi.container.destroy_container(listener_container_name)
    wait_until_container_removed(listener_container_name)
except:
    logger.info("No listener container found.")

logger.info(f"stopped and removed all containers")

Manually check if any "ghost" ports are left behind. Enter their ID (not name) in the next cell and run it to delete them.

In [ ]:
ghost_ports = [
    "281971be-4302-42f0-a3ab-394bff94ff12",

]
for port in ghost_ports:
    try:
    chi.network.delete_port(port)
except:
    logger.info(f"Port {port} not found.")

Proceed to clean up the rest of the project.

In [ ]:
# find the router again
router = None
try:
    router = chi.network.get_router("nils-router")
except Exception as ex:
    logger.info(f"could not find nils-router.")

if router:
    # remove all routes from the router
    chi.network.remove_all_routes_from_router(router["id"])
    logger.success(f"removed all routers from router")

    # remove all subnets from the router
    subnets = chi.network.list_subnets()
    logger.info(f"checking all {len(subnets)} subnets.")
    for subnet in subnets:
        try:
            chi.network.remove_subnet_from_router(router["id"],subnet["id"])
        except Exception as ex:
            pass
    logger.success(f"removed all subnets from router")

    chi.network.delete_router(router["id"])
    logger.success(f"deleted the router")

edgenet = None
try:
    edgenet = chi.network.get_network("nils-edgenet")
except Exception as ex:
    logger.info(f"could not find nils-edgenet.")

if edgenet:
    chi.network.delete_network(edgenet["id"])
    logger.success(f"deleted the nils-edgenet")

Terminate reservations

In [ ]:
leaseslist = list_reservations(brief=True)
for lease in leaseslist:
    unreserve_byid(lease["id"])
    logger.success("Removed " + lease["name"])

logger.info("no leases remaining.")